自前のスコア計算にバグがないか確認  
https://www.kaggle.com/code/cdeotte/compute-validation-score-cv-565/notebook  

In [1]:
import os
import sys
import traceback
import gc
import random
import pickle
import pathlib
import subprocess
from dataclasses import dataclass
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import inspect

import pandas as pd
import numpy as np
import itertools
import cudf
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import line_notify
import my_logger
from noglobal import noglobal

# 設定

In [2]:
@dataclass
class Cfg:
    loglevel = "INFO"
    exp_name = "exp053"
    seed = 42
    k = 20
    cand_n = 15
    negative_sample = 1
    train_chunk_n = 1
    test_chunk_n = 2
    type2id = {"clicks":0, "carts":1, "orders":2}
    id2type = {0:"clicks", 1:"carts", 2:"orders"}
    train_weeks = ["week3"]
    valid_week = "week4"
    valid_session_n = 100_000
    input_dir = os.getenv('INPUT_DIR')
    output_dir = os.getenv('OUTPUT_DIR')
    prep_dir = os.getenv("PREP_DIR")

    clicks_params = {'objective': 'binary', 'boosting': 'gbdt', 'learning_rate': 0.1, 'metric': 'binary_logloss', 'seed': 42, 'feature_pre_filter': False, 'lambda_l1': 5.485903737168179, 'lambda_l2': 0.005594683492536064, 'num_leaves': 79, 'feature_fraction': 0.552, 'bagging_fraction': 0.9295272232672004, 'bagging_freq': 2, 'min_child_samples': 10}
    carts_params = {'objective': 'binary', 'boosting': 'gbdt', 'learning_rate': 0.1, 'metric': 'binary_logloss', 'seed': 42, 'feature_pre_filter': False, 'lambda_l1': 8.709050252544463, 'lambda_l2': 0.06935262036337767, 'num_leaves': 252, 'feature_fraction': 0.4, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 5}
    orders_params = {'objective': 'binary', 'boosting': 'gbdt', 'learning_rate': 0.1, 'metric': 'binary_logloss', 'seed': 42, 'feature_pre_filter': False, 'lambda_l1': 9.356310279757256, 'lambda_l2': 1.3120983078968551e-08, 'num_leaves': 174, 'feature_fraction': 0.5, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}

cfg = Cfg()

In [101]:
@noglobal(excepts=["cfg", "logger"])
def evaluate(clicks_labels, carts_labels, orders_labels, 
             clicks_preds, carts_preds, orders_preds, k=20):

    num_clicks = 0
    num_carts = 0
    num_orders = 0
    hit_clicks = 0
    hit_carts = 0
    hit_orders = 0

    for i in range(len(clicks_labels)):
        clicks_label = clicks_labels[i]
        carts_label = carts_labels[i]
        orders_label = orders_labels[i]
        clicks_pred = clicks_preds[i]
        carts_pred = carts_preds[i]
        orders_pred = orders_preds[i]

        if type(clicks_pred) == list:
            clicks_pred = clicks_pred[:k]
        else:
            clicks_pred = []
        if type(carts_pred) == list:
            carts_pred = carts_pred[:k]
        else:
            carts_pred = []    
        if type(orders_pred) == list:
            orders_pred = orders_pred[:k]
        else:
            orders_pred = []

        if not np.isnan(clicks_label):
            num_clicks += 1
            hit_clicks += int(clicks_label[0] in clicks_pred)

        if type(carts_label) == np.ndarray:
            num_carts += min(len(carts_label), k)
            hit_carts += len(set(carts_pred) & set(carts_label))
            
        if type(orders_label) == np.ndarray:
            num_orders += min(len(orders_label), k)
            hit_orders += len(set(orders_pred) & set(orders_label))


    recall_clicks = hit_clicks / num_clicks
    recall_carts = hit_carts / num_carts
    recall_orders = hit_orders / num_orders
    w_recall_clicks = recall_clicks * 0.10
    w_recall_carts = recall_carts * 0.30
    w_recall_orders = recall_orders * 0.60
    score = w_recall_clicks + w_recall_carts + w_recall_orders

    results = {}
    results["num_clicks"] = num_clicks
    results["hit_clicks"] = hit_clicks
    results["num_carts"] = num_carts
    results["hit_carts"] = hit_carts
    results["num_orders"] = num_orders
    results["hit_orders"] = hit_orders
    results["recall_clicks"] = format(recall_clicks, ".3f")
    results["recall_carts"] = format(recall_carts, ".3f")
    results["recall_orders"] = format(recall_orders, ".3f")
    results["w_recall_clicks"] = format(w_recall_clicks, ".3f")
    results["w_recall_carts"] = format(w_recall_carts, ".3f")
    results["w_recall_orders"] = format(w_recall_orders, ".3f")
    results["score"] = format(score, ".3f")

    return results

In [4]:
pred_df = pd.read_csv(cfg.output_dir + "chris_valid/validation_preds.csv")

In [5]:
score = 0
weights = {'clicks': 0.10, 'carts': 0.30, 'orders': 0.60}
gts = {}
preds = {}

for t in ['clicks','carts','orders']:
    sub = pred_df.loc[pred_df.session_type.str.contains(t)].copy()
    sub['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
    sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])
    test_labels = pd.read_parquet(cfg.output_dir + "chris_valid/test_labels.parquet")
    test_labels = test_labels.loc[test_labels['type']==t]
    test_labels = test_labels.merge(sub, how='left', on=['session'])
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
    recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
    score += weights[t]*recall
    print(f'{t} recall =',recall)
    gts[t] = test_labels["ground_truth"].to_list()
    preds[t] = test_labels["labels"].to_list()

    
print('=============')
print('Overall Recall =',score)
print('=============')

clicks recall = 0.5255597442145808
carts recall = 0.4093328152483512
orders recall = 0.6487936598117477
Overall Recall = 0.5646320148830121


In [48]:
pred_df = pd.read_csv(cfg.output_dir + "chris_valid/validation_preds.csv")
test_labels = pd.read_parquet(cfg.output_dir + "chris_valid/test_labels.parquet")

pred_df.labels = pred_df.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])
test_labels["session_type"] = test_labels["session"].astype(str) + "_" + test_labels["type"] 

pred_df = pred_df.merge(test_labels, on=["session_type"], how="left")

In [100]:
type(pred_df[pred_df["session_type"].str.contains("carts")]["ground_truth"].to_list()[0]) == np.ndarray

True

In [102]:
evaluate(clicks_labels=pred_df[pred_df["session_type"].str.contains("clicks")]["ground_truth"].to_list(), 
         carts_labels=pred_df[pred_df["session_type"].str.contains("carts")]["ground_truth"].to_list(), 
         orders_labels=pred_df[pred_df["session_type"].str.contains("orders")]["ground_truth"].to_list(), 
         clicks_preds=pred_df[pred_df["session_type"].str.contains("clicks")]["labels"].to_list(),
         carts_preds=pred_df[pred_df["session_type"].str.contains("carts")]["labels"].to_list(), 
         orders_preds=pred_df[pred_df["session_type"].str.contains("orders")]["labels"].to_list(), 
         k=20)

{'num_clicks': 1755534,
 'hit_clicks': 922638,
 'num_carts': 576482,
 'hit_carts': 235973,
 'num_orders': 313303,
 'hit_orders': 203269,
 'recall_clicks': '0.526',
 'recall_carts': '0.409',
 'recall_orders': '0.649',
 'w_recall_clicks': '0.053',
 'w_recall_carts': '0.123',
 'w_recall_orders': '0.389',
 'score': '0.565'}

In [78]:
sessions_label

,session,clicks_labels,carts_labels,orders_labels
0,0,536842.0,"[974651, 280978, 1521766, 1760145, 275288, 543...","[1199474, 543308]"
1,1,1628317.0,None,None
2,2,477910.0,None,None
3,4,1838173.0,"[1554752, 917213, 758750]",None
4,6,1292948.0,None,None
...,...,...,...,...
4356192,12899774,1399483.0,None,None
4356193,12899775,1760714.0,None,None
4356194,12899776,1737908.0,None,None
4356195,12899777,384045.0,None,None


In [39]:
df = df.merge(pred_df, on=["session_type"], how="left")
df = df.merge(test_labels, )

,session_type,labels,session
0,11098528_clicks,"[11830, 588923, 1732105, 571762, 884502, 11578...",11098528
1,11098529_clicks,"[1105029, 459126, 1339838, 1544564, 217742, 16...",11098529
2,11098530_clicks,"[409236, 264500, 1603001, 963957, 254154, 5830...",11098530
3,11098531_clicks,"[396199, 1271998, 452188, 1728212, 1365569, 62...",11098531
4,11098532_clicks,"[876469, 7651, 108125, 1202618, 1159379, 77906...",11098532
...,...,...,...
5403748,12899774_carts,"[33035, 1539309, 819288, 95488, 270852, 771913...",12899774
5403749,12899775_carts,"[1743151, 1760714, 1163166, 1255910, 1498443, ...",12899775
5403750,12899776_carts,"[548599, 1401030, 1150130, 1440959, 1144446, 1...",12899776
5403751,12899777_carts,"[384045, 1308634, 395762, 1688215, 703474, 148...",12899777


In [38]:
df

,session
0,11098528_clicks
1,11098529_clicks
2,11098530_clicks
3,11098531_clicks
4,11098532_clicks
...,...
1801246,12899774_orders
1801247,12899775_orders
1801248,12899776_orders
1801249,12899777_orders


In [28]:
df = df.merge(test_labels, on="session", how="left")

In [30]:
df.isnull().sum()

session         0
type            0
ground_truth    0
dtype: int64

In [15]:
test_labels["session"].nunique()

1801251

In [9]:
evaluate(clicks_labels=gts["clicks"],
         carts_labels=gts["carts"],
         orders_labels=gts["orders"], 
         clicks_preds=preds["clicks"], 
         carts_preds=preds["carts"],
         orders_preds=preds["orders"], 
         k=20)

IndexError: list index out of range